In [62]:
import math
import random
import numpy as np
%run TSP.ipynb

In [63]:
drilling_problem_data = TSP('a280.tsp')
print(drilling_problem_data)

NOME : a280
PROBLEMA : drilling problem (Ludwig)
TIPO : TSP
TAMNHO: 280
CALCULO : EUC_2D


In [64]:
def calcular_fitness(individuo, tsp):
    fitness = 0
    for i in range(len(individuo)):
        p1 = tsp.data[str(individuo[i])]
        if i + 1 < len(individuo):
            p2 = tsp.data[str(individuo[i+1])]
        else:
            p2 = tsp.data[str(individuo[0])]
        
        fitness += calcular_distancia(p1, p2)
    return fitness

In [65]:
def calcular_distancia(p1, p2):
    xd = p1[0] - p1[1];
    yd = p2[0] - p2[1];
    dij = int(math.sqrt( xd*xd + yd*yd) );
    return dij

In [66]:
def gerar_solucao(tamanho_dna):
    dna = list(range(1, (tamanho_dna + 1)))
    random.shuffle(dna)
    return dna

In [67]:
def gerar_populacao_aleatoria(tamanho_populacao, tamanho_dna):
    populacao = []
    for indice in range(1, (tamanho_populacao+1)):
        populacao.append({"indice": str(indice), "dna": gerar_solucao(tamanho_dna)})
    return populacao

In [68]:
def get_individuo_por_indice(indice, populacao):
    return list(filter(lambda ind: ind['indice'] == str(indice), populacao))[0]

In [69]:
def crossover(pai, mae):
    # Valores de corte para o DNA
    inicio = random.randint(0, len(pai['dna'])-1)
    fim = random.randint(inicio+1, len(pai['dna']))
    
    # Cria genes iniciais para filho a partir do pai
    dna_filho = pai['dna'][inicio:fim]
    
    # Contabiliza quantos genes faltam
    genes_faltando = len(pai['dna']) - len(dna_filho) - 1
    
    genes_adicionados = 0
    i = 0
    
    # Atribuí genes restantes a partir da mãe
    while genes_adicionados < genes_faltando:
        gene = mae['dna'][i]
        
        if gene not in dna_filho:
            dna_filho.append(gene)
            genes_adicionados += 1
    
        i += 1
    
    # Aponta para ponto inicial
    dna_filho.append(dna_filho[0])
        
    return dna_filho

In [70]:
def mutar(dna, taxaMutacao):
    dna_antigo = dna[:-1]
    
    for i in range(len(dna_antigo)):
        if(random.random() < taxaMutacao):
            j = int(random.random() * len(dna_antigo))
            
            geneA = dna_antigo[i]
            geneB = dna_antigo[j]
            
            dna_antigo[i] = geneB
            dna_antigo[j] = geneA
    
    dna_novo = dna_antigo
    # Aponta para ponto inicial
    dna_novo.append(dna_novo[0])
    return dna_novo

In [71]:
a = [1,2,3,4,5,6,7,8,9,1]
mutar(a, 0.03)

[9, 2, 1, 4, 5, 6, 7, 8, 3, 9]

In [72]:
def algoritmo_genetico(limite_de_geracoes=5000, limite_sem_melhora=500, 
                       tamanho_populacao=100, taxa_cruzamento=0.8, taxa_mutacao=0.02 , 
                       tsp=TSP('eil51.tsp')):
    # Cria uma população aleatóriamente, conforme `tamanho_populacao`.
    # Cada individuo representa uma possível solução
    populacao = gerar_populacao_aleatoria(tamanho_populacao, tsp.tamanho)

    tamanho_populacao_cruzamento = math.ceil(tamanho_populacao * taxa_cruzamento)
    
    indice_total = tamanho_populacao + 1
    
    geracao_atual = 0
    geracoes_sem_melhora = 0
    
    melhor_individuo = None
    
    # geracao_atual < limite_de_geracoes and
    while geracoes_sem_melhora < limite_sem_melhora:
        # Armazena fitness da populução atual
        populacao_fitness = {}

        # Calcula fitness da populução atual
        for individuo in populacao:
            populacao_fitness[individuo['indice']] = calcular_fitness(individuo['dna'], tsp)

        nova_populacao = []
        melhor_individuo_da_geracao = None
        
        # Pega % da população com melhor fitness e cria descendentes a partir deles
        for rank, indice in enumerate(sorted(populacao_fitness, key=populacao_fitness.get)[:tamanho_populacao_cruzamento]):
            individuo = get_individuo_por_indice(indice, populacao)
            
            if rank == 0:
                melhor_individuo_da_geracao = individuo

            # Adiciona individuopara próxima população
            nova_populacao.append(individuo)
            
            # Seleciona mãe aleatoriamente dentro da população
            pai = individuo
            mae = random.choice(populacao)
            
            # Cria filho
            filho = {}
            filho['indice'] = str(indice_total)
            filho['dna'] = mutar(crossover(pai, mae), taxa_mutacao)
            indice_total += 1
            
            # Adiciona filho a nova população
            nova_populacao.append(filho)

        # Substitui população antiga com nova polação de descendentes
        while len(nova_populacao) < len(populacao):
            nova_populacao.append(random.choice(populacao))
        
        populacao = nova_populacao
        
        
        # Ajusta valores para próxima geração
        if melhor_individuo != melhor_individuo_da_geracao:
            melhor_individuo = melhor_individuo_da_geracao
            geracoes_sem_melhora = 0
        else:
            geracoes_sem_melhora += 1
        
        print("Geração: {}\tLimite sem melhora: {}\t Fitness: {}\t Len: {}".format(geracao_atual, geracoes_sem_melhora, calcular_fitness(melhor_individuo['dna'], tsp), len(populacao)))
        
        geracao_atual += 1
    
        
        

In [73]:
algoritmo_genetico()

Geração: 0	Limite sem melhora: 0	 Fitness: 1467	 Len: 160
Geração: 1	Limite sem melhora: 0	 Fitness: 1429	 Len: 160
Geração: 2	Limite sem melhora: 0	 Fitness: 1401	 Len: 160
Geração: 3	Limite sem melhora: 1	 Fitness: 1401	 Len: 160
Geração: 4	Limite sem melhora: 2	 Fitness: 1401	 Len: 160
Geração: 5	Limite sem melhora: 3	 Fitness: 1401	 Len: 160
Geração: 6	Limite sem melhora: 0	 Fitness: 1386	 Len: 160
Geração: 7	Limite sem melhora: 1	 Fitness: 1386	 Len: 160
Geração: 8	Limite sem melhora: 2	 Fitness: 1386	 Len: 160
Geração: 9	Limite sem melhora: 3	 Fitness: 1386	 Len: 160
Geração: 10	Limite sem melhora: 0	 Fitness: 1360	 Len: 160
Geração: 11	Limite sem melhora: 1	 Fitness: 1360	 Len: 160
Geração: 12	Limite sem melhora: 2	 Fitness: 1360	 Len: 160
Geração: 13	Limite sem melhora: 3	 Fitness: 1360	 Len: 160
Geração: 14	Limite sem melhora: 4	 Fitness: 1360	 Len: 160
Geração: 15	Limite sem melhora: 5	 Fitness: 1360	 Len: 160
Geração: 16	Limite sem melhora: 6	 Fitness: 1360	 Len: 160
Geração

IndexError: list index out of range